In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys,pandas,os
sys.path.append('/home/heyangle/Desktop/ScanCraft/ScanCraft')

In [2]:
import sys,numpy,pandas,os,queue
from copy import deepcopy
sys.path.append('/home/heyangle/Desktop/ScanCraft/ScanCraft')
from command.nexus.SPheno import SPheno,block_mapping,ReadSLHAFile
from command.scan.scan import scan
from command.multi_thread.MT_SPheno import MT_SPheno
from command.multi_thread.queue_operation import GenerateQueue # generate random queue
from command.pytorch.normalize import normalize
from command.operators.iterable import FlatToList
from command.data_transformer.ArrayToInputQueue import ArrayToInputQueue
from command.data_transformer.InputToPandas import InputToPandas
from command.data_transformer.defult_parameter_order import defult_name_order
from command.file_operations.GetSamples import GetSamples

from command.scan.GA import ga

In [3]:
# a mold of input point object
mold=scan(method='random')
# MinPar -------
mold.AddScalar('tanB','MINPAR',3,1.,60.)
# Extpar -------
mold.AddScalar('Lambda','EXTPAR', 61, 1e-3, 0.7, prior_distribution='exponential')
mold.AddScalar('Kappa', 'EXTPAR', 62, 1e-3, 0.7, prior_distribution='exponential')
# mold.AddScalar('A_Lambda','EXTPAR' ,63,-3.e3,3.e3)
mold.AddScalar('A_kappa','EXTPAR',64, -5.e2, 5.e2)
mold.AddScalar('mu_eff', 'EXTPAR',65,100., 300.)
# MSoft --------
# mold.AddScalar('M1','MSOFT',1,50,2000)
# mold.AddScalar('M2','MSOFT',2,100,2000)
# Squark -------
# mold.AddElement( 'Mq3^2','MSQ2',(3,3), 1e4,4e6)
# mold.AddElement( 'Mu3^2','MSU2',(3,3), 1e4,4e6)
# mold.AddFollower('Md3^2','MSD2',(3,3), 'Mu32')
mold.AddElement( 'Tt',   'TU',  (3,3), -2000,2000)
mold.AddElement( 'Tb',   'TD',  (3,3), -2000,2000)
# Slepton ------
mold.AddElement( 'Ml3^2','MSL2',(3,3), 1e4,25e4)
mold.AddElement( 'Ml2^2','MSL2',(2,2), 1e4,25e4)
# mold.AddFollower('Ml1^2','MSL2',(1,1),'Ml32')
# mold.AddFollower('Me3^2','MSE2',(3,3),'Ml32')
# mold.AddFollower('Me2^2','MSE2',(2,2),'Ml32')
# mold.AddFollower('Me1^2','MSE2',(1,1),'Ml32')
mold.AddElement( 'Ttau', 'TE',  (3,3),-2000,2000)
# Sneutrino ----
mold.AddElement( 'Mv3^2', 'MV2', (3,3), 1e4, 9e4)
mold.AddElement( 'Mv2^2', 'MV2', (2,2), 1e4,25e4)
mold.AddElement( 'Mx3^2', 'MX2', (3,3), 1e4, 9e4)
mold.AddElement( 'Mx2^2', 'MX2', (2,2), 1e4,25e4)
mold.AddElement( 'Yv3',   'YV',  (3,3), 0, 0.5)
mold.AddElement( 'Tv3',   'TV',  (3,3), -500,500)
mold.AddElement( 'Yv2',   'YV',  (2,2), 0,0.5)
mold.AddElement( 'Tv2',   'TV',  (2,2), -500,500)
mold.AddElement( 'Lambda_v3',  'LAMN', (3,3), 0,  0.5)
mold.AddElement( 'Lambda_v2',  'LAMN', (2,2), 0,  0.5)
mold.AddElement( 'T_Lambda_v3','TLAMN',(3,3), -500,500)
mold.AddElement( 'T_Lambda_v2','TLAMN',(2,2), -500,500)
# muX
for i in range(1,4):
    for j in range(1,4):
        mold.AddDependent(f'muX_{i}{j}','MUX',tuple((i,j)))
# function to get each muX elements
from command.ISS_mu_x import GetMuX as muX
def GetMuX(sample):
    #input: Yv_11, Yv_22, Yv_33, LAMN_11, LAMN_22, LAMN_33, tanbeta, vs
    M=muX(0.01,# Yv_11
        sample.variable_list['Yv2'](),
        sample.variable_list['Yv3'](),
        0.3,# LAMN_11
        sample.variable_list['Lambda_v2'](),
        sample.variable_list['Lambda_v3'](),
        sample.variable_list['Lambda_v2'](),
        sample.variable_list['tanB'](),
        sample.variable_list['tanB']()/sample.variable_list['Lambda']()
    )
    for i in range(1,4):
        for j in range(1,4):
            sample.variable_list[f'muX_{i}{j}'].value=M[i-1,j-1]

order=defult_name_order(mold.free_parameter_list) # parameter name list
dimention=len(order)

Norm=normalize(mold,order=order)

In [4]:
sorted(list(mold.variable_list.keys()))

['A_kappa',
 'Kappa',
 'Lambda',
 'Lambda_v2',
 'Lambda_v3',
 'Ml2^2',
 'Ml3^2',
 'Mv2^2',
 'Mv3^2',
 'Mx2^2',
 'Mx3^2',
 'T_Lambda_v2',
 'T_Lambda_v3',
 'Tb',
 'Tt',
 'Ttau',
 'Tv2',
 'Tv3',
 'Yv2',
 'Yv3',
 'muX_11',
 'muX_12',
 'muX_13',
 'muX_21',
 'muX_22',
 'muX_23',
 'muX_31',
 'muX_32',
 'muX_33',
 'mu_eff',
 'tanB']

In [5]:
mold.variable_list['muX_33'].block

'MUX'

In [6]:
block_mapping

{'MSOFT': 'MSOFTIN',
 'MSQ2': 'MSQ2IN',
 'MSU2': 'MSU2IN',
 'MSD2': 'MSD2IN',
 'MSL2': 'MSL2IN',
 'MSE2': 'MSE2IN',
 'TU': 'TUIN',
 'TD': 'TDIN',
 'TE': 'TEIN',
 'MUX': 'MUXIN',
 'MV2': 'MV2IN',
 'MX2': 'MX2IN',
 'YV': 'YVIN',
 'LAMN': 'LAMNIN',
 'BMUX': 'BMUXIN',
 'TV': 'TVIN',
 'TLAMN': 'TLAMNIN'}

In [7]:
mold.GetValue('LesHouches.in.NInvSeesaw_low',mapping=block_mapping)

In [8]:
GetMuX(mold)

In [9]:
mold.Print()

   A_kappa: -36.0
     Kappa: 0.1
    Lambda: 0.2
 Lambda_v2: 0.1
 Lambda_v3: 0.1
     Ml2^2: 1000000.0
     Ml3^2: 1000000.0
     Mv2^2: 90000.0
     Mv3^2: 90000.0
     Mx2^2: 90000.0
     Mx3^2: 90000.0
T_Lambda_v2: 200.0
T_Lambda_v3: 200.0
        Tb: 10.0
        Tt: 1000.0
      Ttau: 1.0
       Tv2: 0.3
       Tv3: 0.3
       Yv2: 0.5
       Yv3: 0.5
    muX_11: 1.5463757030035157e-08
    muX_12: 1.5556407974296435e-13
    muX_13: 1.6495898986557046e-13
    muX_21: 1.555640797429634e-13
    muX_22: 1.1902059422110807e-14
    muX_23: 7.302063102103514e-16
    muX_31: 1.6495898986557116e-13
    muX_32: 7.302063102103525e-16
    muX_33: 1.208275387669906e-14
    mu_eff: 300.0
      tanB: 10.0


In [10]:
mold.block_list['MUX']

{(1, 1): <command.scan.free_parameter.dependent_scalar at 0x7ffb4b0a80b8>,
 (1, 2): <command.scan.free_parameter.dependent_scalar at 0x7ffb4b0a80f0>,
 (1, 3): <command.scan.free_parameter.dependent_scalar at 0x7ffb4b0a8198>,
 (2, 1): <command.scan.free_parameter.dependent_scalar at 0x7ffb4b0a8208>,
 (2, 2): <command.scan.free_parameter.dependent_scalar at 0x7ffb4b0a8278>,
 (2, 3): <command.scan.free_parameter.dependent_scalar at 0x7ffb4b0a82e8>,
 (3, 1): <command.scan.free_parameter.dependent_scalar at 0x7ffb4b0a8358>,
 (3, 2): <command.scan.free_parameter.dependent_scalar at 0x7ffb4b0a83c8>,
 (3, 3): <command.scan.free_parameter.dependent_scalar at 0x7ffb4b0a8438>}

In [11]:
SP=SPheno(input_mold='LesHouches.in.NInvSeesaw_low'
            ,input_file='LesHouches.in.NInvSeesaw_low'
            ,output_file='SPheno.spc.NInvSeesaw'
            ,main_routine='SPhenoNInvSeesaw')

SPheno file:
->/home/heyangle/Desktop/ScanCraft/ScanCraft/packages/SPheno-4.0.3
delete folder record? (y/n) 



In [12]:
p1=SP.Run(mold)

In [ ]:
p1.